### Set Up

First we import the pertinent libraries

In [1]:
import spacy
import requests
import nltk
import textblob
import csv

from bs4 import BeautifulSoup
from nltk import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In the first line we open the textfile that we would like to analyze.

We load the spacyNLP Named Entity Recognizer as well as the Vader Sentiment Model from NLTK

In [2]:
text = open("washington_times.txt", "r").read()
sent = sent_tokenize(text)

spacyNLP = spacy.load('en_core_web_sm')

sid = SentimentIntensityAnalyzer()

### Analyzing the Data

For each sentence in our data we check if it contains any named entities that are actual persons. 
If there is only one "person" named entity in the sentence then we extract that entity and find their political affiliation on Wikipedia. 

Lastly we run sentiment analysis on this sentence and write all this information to a spreadsheet.

In [3]:
for i in sent:
    j = spacyNLP(i)
    persons = 0
    for element in j.ents:
        if(element.label_ == 'PERSON'):
            persons += 1
    if(persons > 1):
        continue
    
    for element in j.ents:
        if(element.label_ == 'PERSON'):
            x = element.text.split()
            if(len(x) == 2):
                page = requests.get("https://en.wikipedia.org/wiki/" + x[0] + "_" + x[1])
                soup = BeautifulSoup(page.text, 'html.parser')
                table = soup.find('table')
                if table is None:
                    continue
                party = table.find('th', string="Political party")
                if(party is not None):
                    ss = sid.polarity_scores(i)
                    results = x[0] + " " + x[1] + "," + party.find_next_sibling().text.split()[0] + "," + str(ss["compound"]) + "\n"
                    with open('results_american_conservative.csv', "a") as out:
                        out.write(results)